# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print("current working directory:", os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print("the filepath to the data:", filepath)

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    
# uncomment the code below if you would like to see a list of all files' paths
#print(file_path_list)

current working directory: /home/workspace
the filepath to the data: /home/workspace/event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("total number of rows:",len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

total number of rows: 8056


In [4]:
# check the number of rows in your csv file. This file will be used to create a dernormalized dataset 
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("number of rows in the CSV file:", sum(1 for line in f))

number of rows in the CSV file: 6821


# Part II. Building the Apache Cassandra data model. 

## The CSV file <font color=blue>event_datafile_new.csv</font> is used for inserting data into the tables of the modeled database. This CSV file contains the following columns: 
- artist
- firstName (of user)
- gender (of user)
- itemInSession (item number in session)
- lastName (of user)
- length (of the song)
- level (paid or free song)
- location (of the user)
- sessionId
- song (title)
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=blue>event_datafile_new.csv</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance on your local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Create a session to establish connection and begin executing queries
session = cluster.connect()

#### Creating Keyspace

In [6]:
#One needs to create a keyspace where he/she will work in

try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#After creation a keyspace, one needs to connect to this keyspace

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Data model will be based on the queries which answer the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Query 1: Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4

# Create a table 'artist_song_data' with required fields needed to fulfil the #1 query
# As a primary key, the table uses the columns 'sessionId' and 'itemInSession  which are a partition key and a clustering key respectively
# The partition of data is done by using 'sessionId', and within this partition the rows will be ordered by 'itemInSession' in ascending order

query = "CREATE TABLE IF NOT EXISTS udacity.artist_song_data"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, \
                PRIMARY KEY(sessionId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# Here the created dataset stored into the 'event_datafile_new.csv' file will be used and set up.
# Using INSERT statement the data will be inserted into an 'artist_song_data' table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO udacity.artist_song_data (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verify that the data have been inserted into `artist_song_data` table

In [10]:
query = "SELECT artist, song, length \
        FROM artist_song_data \
        WHERE sessionId=338 AND itemInSession=4"

try:
    rows = session.execute(query)
    df1 = pd.DataFrame(list(rows))
except Exception as e:
    print(e)
df1

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


In [11]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

# Create a table 'artist_song_user_data' with required fields needed to fulfil the #2 query
# This table uses the 'userId' and 'sessionId' columns as a composition partition key in the primary key. 
# Using a composite partition key helps to break data into smaller chunks grouped over userId and sessionId.
# The partitioned data is sorted by itemInSession in ascending order.

query = "CREATE TABLE IF NOT EXISTS udacity.artist_song_user_data"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, \
                PRIMARY KEY((userId,sessionId), itemInSession))"
session.execute(query)

# the 'event_datafile_new.csv' file will be used to extract data from created dataset and \
#  insert it into an 'artist_song_user_data' table.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO udacity.artist_song_user_data (userId, sessionId, artist, song, itemInSession, firstName, lastName)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))

### Verify that the data have been inserted into `artist_song_user_data` table

In [12]:
query = "SELECT artist, song, firstName, lastName \
        FROM artist_song_user_data \
        WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(query)
    df2 = pd.DataFrame(list(rows))
except Exception as e:
    print(e)
df2

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [13]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Create a table 'user_song_data' with required fields needed to fulfil the #3 query
# This table uses the 'song' and 'userId' columns as a primary key. 
# The data partitioned by partition key 'song' and ordered within a partition by clustering key 'userId' in ascending order.

query = "CREATE TABLE IF NOT EXISTS udacity.user_song_data"
query = query + "(song text, userId int, sessionId int, itemInSession int, firstName text, lastName text, \
                PRIMARY KEY(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# the 'event_datafile_new.csv' file will be used to extract data from created dataset and \
#  insert them into an 'user_song_data' table.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO udacity.user_song_data (song, userId, sessionId, itemInSession, firstName, lastName)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), int(line[8]), int(line[3]), line[1], line[4]))

### Verify that the data have been inserted into `user_song_data` table

In [14]:
query = "SELECT firstName, lastName, userId \
        FROM user_song_data \
        WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
    df3 = pd.DataFrame(list(rows))
except Exception as e:
    print(e)
df3  

,firstname,lastname,userid
0,Jacqueline,Lynch,29
1,Tegan,Levine,80
2,Sara,Johnson,95


### Drop the tables before closing out the sessions

In [15]:
query = "drop table artist_song_data"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table artist_song_user_data"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_song_data"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()